In [3]:
# import sys
# !{sys.executable} -m pip install thefuzz


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
from IPython.display import display
import seaborn as sns # Often good for statistical plots like box plots

import ollama
from thefuzz import process
from thefuzz import fuzz
import time


In [2]:
# now use the Form MA data
# df_form_ma_a = pd.read_csv(r"C:\Users\barid\Documents\Papers\Muni_adv_PBF\Data\output_step_3k.csv", index_col=False)
df_form_ma_a = pd.read_csv("https://www.dropbox.com/scl/fi/psms2ubtcwpa0zd7h4f48/sec_form_ma_a.csv?rlkey=3s2a6wu2g8jxmnw5mwnibchpp&st=3zt050lj&dl=1", index_col=False)
df_form_ma_a

df_form_ma_a['Date_Datetime'] = pd.to_datetime(df_form_ma_a['filing_date'], format='%m-%d-%Y')
df_form_ma_a['yr_filing'] = df_form_ma_a['Date_Datetime'].dt.year
df_form_ma_a['qtr_filing'] = df_form_ma_a['Date_Datetime'].dt.quarter
df_form_ma_a['yr_qtr_filing'] = df_form_ma_a['yr_filing'].astype(str) + 'Q' + df_form_ma_a['qtr_filing'].astype(str)
df_form_ma_a


,firm_name,office_locations,other_business_activities,compensation_agreements,solicitation_activities,filing_date,file_name,folder_name,header_data_submissionType,header_data_filerId,...,regulatory_disclosures_regulatoryDisclosure_isAuthorizedToActAttorney,regulatory_disclosures_regulatoryDisclosure_isRegulatoryComplaint,regulatory_disclosures_civilDisclosure_isEnjoined,regulatory_disclosures_civilDisclosure_isFoundInViolationOfRegulation,regulatory_disclosures_civilDisclosure_isDismissed,regulatory_disclosures_civilDisclosure_isNamedInCivilProceeding,Date_Datetime,yr_filing,qtr_filing,yr_qtr_filing
0,ROBERT W. BAIRD & CO. INCORPORATED,"[{'type': 'principal', 'street1': '777 E. WISC...","['Advice Insurance Of MS', 'Advice Investment ...","['Hourly Charges', 'Fixed Fees', 'Contingent F...",['Not Applicable'],07-11-2014,MA_0000009211_14_000018.txt,9211,MA,9211,...,N,N,N,N,N,N,2014-07-11,2014,3,2014Q3
1,ROBERT W. BAIRD & CO. Inc,"[{'type': 'principal', 'street1': '777 E. WISC...","['Advice Insurance Of MS', 'Advice Investment ...","['Hourly Charges', 'Fixed Fees', 'Contingent F...",['Not Applicable'],03-23-2015,MA_0000009211_15_000011.txt,9211,MA-A,9211,...,N,N,N,N,N,N,2015-03-23,2015,1,2015Q1
2,ROBERT W. BAIRD & CO. INCORPORATED,"[{'type': 'principal', 'street1': '777 E. WISC...","['Advice Insurance Of MS', 'Advice Investment ...","['Hourly Charges', 'Fixed Fees', 'Contingent F...",['Not Applicable'],06-25-2015,MA_0000009211_15_000026.txt,9211,MA/A,9211,...,N,N,N,N,N,N,2015-06-25,2015,2,2015Q2
3,ROBERT W. BAIRD & CO. INCORPORATED,"[{'type': 'principal', 'street1': '777 E. WISC...","['Advice Insurance Of MS', 'Advice Investment ...","['Hourly Charges', 'Fixed Fees', 'Contingent F...",['Not Applicable'],03-28-2016,MA_0000009211_16_000043.txt,9211,MA-A,9211,...,N,N,N,N,N,N,2016-03-28,2016,1,2016Q1
4,ROBERT W. BAIRD & CO. INCORPORATED,"[{'type': 'principal', 'street1': '777 E. WISC...","['Advice Insurance Of MS', 'Advice Investment ...","['Hourly Charges', 'Fixed Fees', 'Contingent F...",['Not Applicable'],05-04-2016,MA_0000009211_16_000048.txt,9211,MA/A,9211,...,N,N,N,N,N,N,2016-05-04,2016,2,2016Q2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7858,"First Security Municipal Advisors, Inc.","[{'type': 'principal', 'street1': '501 PRESIDE...","['Advice Insurance Of MS', 'Advice Investment ...","['Hourly Charges', 'Fixed Fees', 'Contingent F...",['Not Applicable'],12-31-2024,MA_0002041221_24_000006.txt,2041221,MA,2041221,...,N,N,N,N,N,N,2024-12-31,2024,4,2024Q4
7859,"First Security Municipal Advisors, Inc.","[{'type': 'principal', 'street1': '501 PRESIDE...","['Advice Insurance Of MS', 'Advice Investment ...","['Hourly Charges', 'Fixed Fees', 'Contingent F...",['Not Applicable'],01-16-2025,MA_0002041221_25_000015.txt,2041221,MA/A,2041221,...,N,N,N,N,N,N,2025-01-16,2025,1,2025Q1
7860,"First Security Municipal Advisors, Inc.","[{'type': 'principal', 'street1': '501 PRESIDE...","['Advice Insurance Of MS', 'Advice Investment ...","['Hourly Charges', 'Fixed Fees', 'Contingent F...",['Not Applicable'],02-06-2025,MA_0002041221_25_000017.txt,2041221,MA/A,2041221,...,N,N,N,N,N,N,2025-02-06,2025,1,2025Q1
7861,Patriot Advisors LLC,[],"['Advice Insurance Of MS', 'Advice Investment ...",['Hourly Charges'],['Not Applicable'],02-13-2025,MA_0002053271_25_000007.txt,2053271,MA,2053271,...,N,N,N,N,N,N,2025-02-13,2025,1,2025Q1


In [3]:
lhs_df = df_form_ma_a[['firm_name','filing_date', 'file_name','header_data_submissionType','header_data_filerId']]
lhs_df['date'] = pd.to_datetime(lhs_df['filing_date'], format='%m-%d-%Y')
lhs_df

/var/folders/3s/hndc56bd6k754n21bbk34pwc0000gn/T/ipykernel_818/2561403159.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lhs_df['date'] = pd.to_datetime(lhs_df['filing_date'], format='%m-%d-%Y')


,firm_name,filing_date,file_name,header_data_submissionType,header_data_filerId,date
0,ROBERT W. BAIRD & CO. INCORPORATED,07-11-2014,MA_0000009211_14_000018.txt,MA,9211,2014-07-11
1,ROBERT W. BAIRD & CO. Inc,03-23-2015,MA_0000009211_15_000011.txt,MA-A,9211,2015-03-23
2,ROBERT W. BAIRD & CO. INCORPORATED,06-25-2015,MA_0000009211_15_000026.txt,MA/A,9211,2015-06-25
3,ROBERT W. BAIRD & CO. INCORPORATED,03-28-2016,MA_0000009211_16_000043.txt,MA-A,9211,2016-03-28
4,ROBERT W. BAIRD & CO. INCORPORATED,05-04-2016,MA_0000009211_16_000048.txt,MA/A,9211,2016-05-04
...,...,...,...,...,...,...
7858,"First Security Municipal Advisors, Inc.",12-31-2024,MA_0002041221_24_000006.txt,MA,2041221,2024-12-31
7859,"First Security Municipal Advisors, Inc.",01-16-2025,MA_0002041221_25_000015.txt,MA/A,2041221,2025-01-16
7860,"First Security Municipal Advisors, Inc.",02-06-2025,MA_0002041221_25_000017.txt,MA/A,2041221,2025-02-06
7861,Patriot Advisors LLC,02-13-2025,MA_0002053271_25_000007.txt,MA,2053271,2025-02-13


In [4]:
# rhs_df = pd.read_csv(r"C:\Users\barid\Documents\Papers\Muni_adv_PBF\Data\sec_form_ma_w.csv", index_col=False)
rhs_df = pd.read_csv("https://www.dropbox.com/scl/fi/l9hatixp3tmdx00mz7yvf/sec_form_ma_w.csv?rlkey=q13yevqmnujj4yrvdbv1ijefc&st=dvw2erx6&dl=1", index_col=False)
rhs_df


,submissionType,filerId,filerCcc,filerFileNumber,contactName,contactPhoneNumber,contactEmail,notificationEmails,filing_date_xml,filed_as_of_date_header,...,booksAndRecords_locationCity,booksAndRecords_locationStateOrCountry,booksAndRecords_locationZipCode,booksAndRecords_locationPhoneNumber,booksAndRecords_description,execution_signature,execution_signerName,execution_title,file_name,folder_name
0,MA-W,12518,XXXXXXXX,867-00476,Michael E Robbins,3123645319,mrobbins@williamblair.com,"['eyeomans@williamblair.com', 'kwagner@william...",09-25-2017,20170925,...,Chicago,IL,60606,3122361600,All of William Blair's municipal advisory book...,Kenneth L. Wagner,Kenneth L. Wagner,Partner & Chief Compliance Officer,MA_0000012518_17_000049.txt,12518
1,MA-W,12933,XXXXXXXX,867-01153,Glenn Essert,6108325309,gessert@boenninginc.com,['gessert@boenninginc.com'],06-17-2022,20220617,...,West Conshohocken,PA,19428,61083212112,"Invoices, engagement letters and Municipal Adv...",Glenn Edward Essert,Glenn Edward Essert,Chief Compliance Officer,MA_0000012933_22_000012.txt,12933
2,MA-W,25142,XXXXXXXX,867-01669,Peter Mirsky,3038631900,peter@coughlinandcompany.com,['peter@coughlinandcompany.com'],08-08-2024,20240808,...,Denver,CO,80203,3038631900,"Old tax returns, entity documents, files for a...",Peter Mirsky,Peter Mirsky,CFO,MA_0000025142_24_000003.txt,25142
3,MA-W,36528,XXXXXXXX,867-00349,Paul D. Brown,3098293311,paul@firstmidstate.com,['pat@firstmidstate.com'],07-05-2023,20230705,...,Bloomington,IL,61701,3098306697,all records,Paul D. Brown,Paul D. Brown,President,MA_0000036528_23_000004.txt,36528
4,MA-W,49709,XXXXXXXX,867-01097,Jacob Bongard,612-671-0527,jacob.x.bongard@ampf.com,['jacob.x.bongard@ampf.com'],08-29-2017,20170829,...,EAGAN,MN,55121-2203,651-452-6515,IRON MOUNTAIN STORES CERTAIN BOOKS AND RECORDS...,Elizabeth Hansen,Elizabeth Hansen,Vice President & Chief Compliance Officer,MA_0000049709_17_000039.txt,49709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,MA-W,1956887,XXXXXXXX,867-02652,NICOLAS NORBOGE,9842915825,nnorboge@daneconsultingpartners.com,['nnorboge@daneconsultingpartners.com'],01-03-2025,20250103,...,Durham,NC,27713,979-739-6463,All records and books are kept at the home off...,NICOLAS D NORBOGE,NICOLAS D NORBOGE,Managing Principal and Chief Compliance Officer,MA_0001956887_25_000001.txt,1956887
446,MA-W,1973850,XXXXXXXX,867-02640,Joe Crowley,213-700-2757,joecrowley@pacificpfa.com,['joecrowley@pacificpfa.com'],10-06-2023,20231006,...,ALTADENA,CA,91001,213-700-2757,Files,Joseph Crowley,Joseph Crowley,Principal,MA_0001973850_23_000005.txt,1973850
447,MA-W,1977510,XXXXXXXX,867-02644,Katherine L Clupper,2158503942,klclupper@gmail.com,['klclupper@gmail.com'],04-26-2024,20240426,...,Philadelphia,PA,19130,2158503942,"LLC Incorporation documents, bank documents, a...",Katherine L Clupper,Katherine L Clupper,Owner,MA_0001977510_24_000003.txt,1977510
448,MA-W,2006358,XXXXXXXX,867-02678,Smiley Arthur Chester III,2819740467,acs716@gmail.com,[],NaN,20241121,...,Humble,TX,77396,2819740467,"Formation documents, bank account records",NaN,NaN,NaN,MA_0002006358_24_000009.txt,2006358


In [5]:
rhs_df2 = rhs_df[['submissionType','filed_as_of_date_header','fullLegalName','fileNumber_formData','file_name','filerId']]
rhs_df2

# rename column names to align better
col_map = {'submissionType':'header_data_submissionType', \
                                           'filed_as_of_date_header':'filing_date', \
                                           'fullLegalName':'firm_name', \
                                           'filerId':'header_data_filerId'}
rhs_df3 = rhs_df2.copy().rename(columns = col_map)
rhs_df3

rhs_df3['date'] = pd.to_datetime(rhs_df3['filing_date'], format='%Y%m%d')
rhs_df3

,header_data_submissionType,filing_date,firm_name,fileNumber_formData,file_name,header_data_filerId,date
0,MA-W,20170925,WILLIAM BLAIR & COMPANY L.L.C.,867-00476,MA_0000012518_17_000049.txt,12518,2017-09-25
1,MA-W,20220617,"BOENNING & SCATTERGOOD, INC.",867-01153,MA_0000012933_22_000012.txt,12933,2022-06-17
2,MA-W,20240808,COUGHLIN & COMPANY INC.,867-01669,MA_0000025142_24_000003.txt,25142,2024-08-08
3,MA-W,20230705,FIRST MIDSTATE INC.,867-00349,MA_0000036528_23_000004.txt,36528,2023-07-05
4,MA-W,20170829,"AMERIPRISE FINANCIAL SERVICES, INC.",867-01097,MA_0000049709_17_000039.txt,49709,2017-08-29
...,...,...,...,...,...,...,...
445,MA-W,20250103,"Dane Consulting Partners, LLC",867-02652,MA_0001956887_25_000001.txt,1956887,2025-01-03
446,MA-W,20231006,"Pacific Public Finance Advisors, LLC",867-02640,MA_0001973850_23_000005.txt,1973850,2023-10-06
447,MA-W,20240426,"KLC Financial Advisors, LLC",867-02644,MA_0001977510_24_000003.txt,1977510,2024-04-26
448,MA-W,20241121,Smiley Arthur Chester III,867-02678,MA_0002006358_24_000009.txt,2006358,2024-11-21


In [6]:
vert_combi = pd.concat([lhs_df,rhs_df3])
vert_combi


,firm_name,filing_date,file_name,header_data_submissionType,header_data_filerId,date,fileNumber_formData
0,ROBERT W. BAIRD & CO. INCORPORATED,07-11-2014,MA_0000009211_14_000018.txt,MA,9211,2014-07-11,NaN
1,ROBERT W. BAIRD & CO. Inc,03-23-2015,MA_0000009211_15_000011.txt,MA-A,9211,2015-03-23,NaN
2,ROBERT W. BAIRD & CO. INCORPORATED,06-25-2015,MA_0000009211_15_000026.txt,MA/A,9211,2015-06-25,NaN
3,ROBERT W. BAIRD & CO. INCORPORATED,03-28-2016,MA_0000009211_16_000043.txt,MA-A,9211,2016-03-28,NaN
4,ROBERT W. BAIRD & CO. INCORPORATED,05-04-2016,MA_0000009211_16_000048.txt,MA/A,9211,2016-05-04,NaN
...,...,...,...,...,...,...,...
445,"Dane Consulting Partners, LLC",20250103,MA_0001956887_25_000001.txt,MA-W,1956887,2025-01-03,867-02652
446,"Pacific Public Finance Advisors, LLC",20231006,MA_0001973850_23_000005.txt,MA-W,1973850,2023-10-06,867-02640
447,"KLC Financial Advisors, LLC",20240426,MA_0001977510_24_000003.txt,MA-W,1977510,2024-04-26,867-02644
448,Smiley Arthur Chester III,20241121,MA_0002006358_24_000009.txt,MA-W,2006358,2024-11-21,867-02678


In [7]:
# form_ma_combi = vert_combi[['firm_name','header_data_filerId','filing_date']]
form_ma_combi = vert_combi[['firm_name','header_data_filerId']]
form_ma_combi = form_ma_combi.drop_duplicates(subset = ['firm_name','header_data_filerId'])
form_ma_combi

,firm_name,header_data_filerId
0,ROBERT W. BAIRD & CO. INCORPORATED,9211
1,ROBERT W. BAIRD & CO. Inc,9211
26,WILLIAM BLAIR & COMPANY L.L.C.,12518
32,"BOENNING & SCATTERGOOD, INC.",12933
37,"CARTY & COMPANY, INC.",18026
...,...,...
134,"ENTREX CAPITAL MARKET, LLC",1510788
221,CHALDON ASSOCIATES LLC,1620194
231,"Providence Engineering & Design, LLC",1620468
287,"Catlin Financial, LLC",1623229


In [8]:
# dropbox_link1 = "https://www.dropbox.com/scl/fi/cqhash1qptv4adpcp8emm/issueinfo.dta?rlkey=expd6ig8urxcjmpmz0ovr2ioq&st=mj1bug0f&dl=1"
dropbox_link1 = "https://www.dropbox.com/scl/fi/a2jyuk5t2tbbqxgwl82iu/issuagnt.dta?rlkey=b5x9984xwqgcic1kg632ch86n&st=ep1fehfm&dl=1"

try:
    reader = pd.read_stata(dropbox_link1, iterator=True)
    # print("Attributes and methods of the StataReader object:")
    # print(dir(reader))
    column_names = list(reader.variable_labels().keys())    
    print("Column Names:")
    print(column_names)
    reader.close()
except Exception as e:
    print(f"Error creating StataReader object: {e}")

Column Names:
['issue_id_l', 'agent_id_l', 'agent_role_c']


/var/folders/3s/hndc56bd6k754n21bbk34pwc0000gn/T/ipykernel_818/3939521072.py:11: FutureWarning: The StataReader.close() method is not part of the public API and will be removed in a future version without notice. Using StataReader as a context manager is the only supported method.
  reader.close()


In [9]:
dropbox_link1 = "https://www.dropbox.com/scl/fi/tr287rjqlfveo9owefi20/agent.dta?rlkey=tcza9dixls7orevspv64mnuik&st=bgs3p4gp&dl=1"

try:
    reader = pd.read_stata(dropbox_link1, iterator=True)
    # print("Attributes and methods of the StataReader object:")
    # print(dir(reader))
    column_names = list(reader.variable_labels().keys())    
    print("Column Names:")
    print(column_names)
    reader.close()
except Exception as e:
    print(f"Error creating StataReader object: {e}")

Column Names:
['agent_id_l', 'legal_name_c']


/var/folders/3s/hndc56bd6k754n21bbk34pwc0000gn/T/ipykernel_818/3655506769.py:10: FutureWarning: The StataReader.close() method is not part of the public API and will be removed in a future version without notice. Using StataReader as a context manager is the only supported method.
  reader.close()


In [10]:
df1 = pd.read_stata('https://www.dropbox.com/scl/fi/a2jyuk5t2tbbqxgwl82iu/issuagnt.dta?rlkey=b5x9984xwqgcic1kg632ch86n&st=ep1fehfm&dl=1')
df1

,issue_id_l,agent_id_l,agent_role_c
0,4002,8371.0,BDCL1
1,4002,21778.0,EXCHG
2,4002,21778.0,PAAG1
3,4002,21778.0,REGIS
4,4002,21778.0,TENAG
...,...,...,...
2402346,1955460,30246.0,UNDER
2402347,1955460,50115.0,FNAD1
2402348,1955483,NaN,
2402349,1955486,NaN,


In [11]:
df_ma = df1[(df1['agent_role_c']=='FNAD1') | (df1['agent_role_c']=='FNAD2') ]
df_ma

,issue_id_l,agent_id_l,agent_role_c
243,4072,53692.0,FNAD1
261,4078,27311.0,FNAD1
272,4080,52996.0,FNAD1
321,4103,24926.0,FNAD1
331,4105,24926.0,FNAD1
...,...,...,...
2402329,1955453,49911.0,FNAD1
2402333,1955454,49935.0,FNAD1
2402339,1955457,49935.0,FNAD1
2402343,1955459,62539.0,FNAD1


In [12]:
df_ma2 = df_ma.drop_duplicates(subset=['agent_id_l'])
df_ma2

,issue_id_l,agent_id_l,agent_role_c
243,4072,53692.0,FNAD1
261,4078,27311.0,FNAD1
272,4080,52996.0,FNAD1
321,4103,24926.0,FNAD1
413,4144,27204.0,FNAD1
...,...,...,...
2383937,1934368,68857.0,FNAD1
2387247,1935275,78097.0,FNAD1
2391138,1943360,73153.0,FNAD1
2398177,1949554,66502.0,FNAD1


In [13]:
agent_info = pd.read_stata("https://www.dropbox.com/scl/fi/tr287rjqlfveo9owefi20/agent.dta?rlkey=tcza9dixls7orevspv64mnuik&st=bgs3p4gp&dl=1")
agent_info

,agent_id_l,legal_name_c
0,1,1ST SOURCE BANK
1,2,1ST SUMMIT BANK
2,5,BAD AGENT A PROFFESSIONAL CORP
3,6,A JEAN STAMM
4,7,BALLARD SPAHR ANDREWS AND INGERSOLL LLP
...,...,...
9485,78180,TRADITIONAL BANK
9486,78181,MERCHANTS AND FARMERS BANK
9487,78187,COMMONWEALTH INVESTMENT MANAGEMENT
9488,78188,JOHN F SALOPEK


In [14]:
ma_mergent = df_ma2.merge(agent_info, on = 'agent_id_l', how = 'left' )
ma_mergent = ma_mergent.drop(['agent_role_c','issue_id_l'], axis=1)
ma_mergent

,agent_id_l,legal_name_c
0,53692.0,ROSS SINCLAIRE AND ASSOCIATES INC
1,27311.0,GEORGE K BAUM AND CO
2,52996.0,AIKIN GROUP
3,24926.0,FIRST SOUTHWEST CO
4,27204.0,ARTHURS LESTRANGE AND CO INC
...,...,...
2229,68857.0,ZIONS BANK NA
2230,78097.0,PHOENIX ADVISOR A DIVISION OF FIRST SECURITY M...
2231,73153.0,THE BAKER GROUP
2232,66502.0,BARCLAYS


In [15]:
# --- Sample Data (Replace with your actual table data) ---
table1_names = [
    {"id": "T1_001", "name": "John Doe"},
    {"id": "T1_002", "name": "Jane Smith"},
    {"id": "T1_003", "name": "Michael Johnson"},
    {"id": "T1_004", "name": "Emily White"},
    {"id": "T1_005", "name": "Christopher Brown"},
    {"id": "T1_006", "name": "Maria Garcia"},
    {"id": "T1_007", "name": "David Lee"},
    {"id": "T1_008", "name": "Sarah Miller"},
    {"id": "T1_009", "name": "William Davis"},
    {"id": "T1_010", "name": "Olivia Wilson"}
]

table2_names = [
    {"id": "T2_A", "name": "Jon Do"},
    {"id": "T2_B", "name": "Jane Smythe"},
    {"id": "T2_C", "name": "Micheal Johnsons"},
    {"id": "T2_D", "name": "Emly Whyte"},
    {"id": "T2_E", "name": "Chrisopher Brouwn"},
    {"id": "T2_F", "name": "Marya Garsia"},
    {"id": "T2_G", "name": "Dave Lee"},
    {"id": "T2_H", "name": "Sara Miler"},
    {"id": "T2_I", "name": "Willm Davies"},
    {"id": "T2_J", "name": "Oliva Wlson"},
    {"id": "T2_X", "name": "Non-matching Name X"} # An example of a name that shouldn't match
]

print("--- Starting Crosswalk Process ---")
print(f"Table 1 entries: {len(table1_names)}")
print(f"Table 2 entries: {len(table2_names)}\n")


--- Starting Crosswalk Process ---
Table 1 entries: 10
Table 2 entries: 11



In [16]:
type(table1_names)

list

In [17]:
# # Rename columns to desired field names
# df_temp1 = form_ma_combi.rename(columns={'header_data_filerId': 'id', 'firm_name': 'name'})

# # Then get namedtuples with new names as fields
# table1_names = list(df_temp1.itertuples(index=False))
# print(table1_names[0].name)  # Access by new field name

# df_temp2 = ma_mergent.rename(columns={'agent_id_l': 'id', 'legal_name_c': 'name'})

# # Then get namedtuples with new names as fields
# table2_names = list(df_temp2.itertuples(index=False))
# print(table2_names[0].name)  # Access by new field name

# Rename columns to desired field names
df_temp1 = form_ma_combi.rename(columns={'header_data_filerId': 'id', 'firm_name': 'name'})
df_temp2 = ma_mergent.rename(columns={'agent_id_l': 'id', 'legal_name_c': 'name'})

# Convert to list of dicts (like your sample data)
table1_names = df_temp1[['id', 'name']].to_dict(orient='records')
table2_names = df_temp2[['id', 'name']].to_dict(orient='records')

# Example prints to verify
print(table1_names[0]['name'])
print(table2_names[0]['name'])

print("--- Starting Crosswalk Process ---")
print(f"Table 1 entries: {len(table1_names)}")
print(f"Table 2 entries: {len(table2_names)}\n")



ROBERT W. BAIRD & CO. INCORPORATED
ROSS SINCLAIRE AND ASSOCIATES INC
--- Starting Crosswalk Process ---
Table 1 entries: 1028
Table 2 entries: 2234



In [29]:

# --- Configuration ---
OLLAMA_MODEL = "mistral"  # Ensure you have 'mistral' pulled in Ollama
FUZZY_MATCH_THRESHOLD = 75 # Initial fuzzy matching threshold (0-100)
LLM_CONFIDENCE_THRESHOLD = 0.8 # Threshold for LLM's confidence score (0.0-1.0)
MAX_LLM_CANDIDATES = 5 # How many top fuzzy matches to send to the LLM for re-evaluation


# --- Ollama Interaction Function ---
def get_llm_match_score(name1: str, name2: str) -> float:
    """
    Uses the Ollama LLM (Mistral) to determine a similarity score between two names.
    The LLM is asked to output a JSON object with a 'match_score'.
    """
    prompt = f"""You are an expert at identifying similar names, even with spelling errors.
    Given two names, rate their similarity on a scale from 0.0 to 1.0, where 1.0 is a perfect match
    and 0.0 is no similarity. Consider common spelling mistakes and variations.

    Return your answer as a JSON object with a single key 'match_score'.
    Example: {{ "match_score": 0.95 }}

    Name 1: "{name1}"
    Name 2: "{name2}"
    """
    
    # Implement exponential backoff for API calls
    retries = 3
    for i in range(retries):
        try:
            response = ollama.chat(
                model=OLLAMA_MODEL,
                messages=[{'role': 'user', 'content': prompt}],
                options={'temperature': 0.1}, # Keep temperature low for deterministic output
                format='json' # Request JSON output
            )
            # Parse the JSON response
            import json
            if 'message' in response and 'content' in response['message']:
                content = response['message']['content']
                try:
                    score_data = json.loads(content)
                    return float(score_data.get('match_score', 0.0))
                except json.JSONDecodeError:
                    print(f"Warning: LLM did not return valid JSON for '{name1}' vs '{name2}'. Response: {content}")
                    return 0.0 # Default to no match if JSON is invalid
            else:
                print(f"Warning: Unexpected LLM response structure for '{name1}' vs '{name2}'. Response: {response}")
                return 0.0
        except ollama.ResponseError as e:
            if e.status_code == 404:
                print(f"Error: Model '{OLLAMA_MODEL}' not found. Please pull it using 'ollama pull {OLLAMA_MODEL}'.")
                return 0.0
            elif e.status_code == 503 and i < retries - 1: # Service Unavailable, retry
                wait_time = (2 ** i) # Exponential backoff
                print(f"LLM call failed (status 503), retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"Error communicating with Ollama: {e}")
                return 0.0
        except Exception as e:
            print(f"An unexpected error occurred during LLM call: {e}")
            return 0.0
    return 0.0 # Return 0 if all retries fail


# --- Perform Crosswalk ---
crosswalk_results = []

# Extract just the names for fuzzy matching for efficiency
table2_names_only = [entry["name"] for entry in table2_names]
table2_id_map = {entry["name"]: entry["id"] for entry in table2_names}


for entry1 in table1_names:
    name1 = entry1["name"]
    id1 = entry1["id"]

    best_fuzzy_match_name = None
    best_fuzzy_match_id = None
    best_fuzzy_score = -1

    # Step 1: Initial fuzzy matching to get top candidates
    fuzzy_matches = process.extract(name1, table2_names_only, scorer=fuzz.token_sort_ratio, limit=MAX_LLM_CANDIDATES)

    llm_determined_match_name = None
    llm_determined_match_id = None
    highest_llm_score = 0.0

    if not fuzzy_matches:
        print(f"No fuzzy matches found for '{name1}' (ID: {id1}).")
        crosswalk_results.append({
            "Table1_ID": id1,
            "Table1_Name": name1,
            "Table2_ID_Match": None,
            "Table2_Name_Match": None,
            "Match_Method": "No Match",
            "Fuzzy_Score": None,
            "LLM_Score": None
        })
        continue

    # Step 2: Re-evaluate top fuzzy matches using LLM for semantic understanding
    for candidate_name, fuzzy_score in fuzzy_matches:
        if fuzzy_score >= FUZZY_MATCH_THRESHOLD:
            llm_score = get_llm_match_score(name1, candidate_name)
            if llm_score > highest_llm_score:
                highest_llm_score = llm_score
                llm_determined_match_name = candidate_name
                llm_determined_match_id = table2_id_map.get(candidate_name)
                best_fuzzy_match_name = candidate_name # Store the fuzzy match that led to the best LLM score
                best_fuzzy_match_id = table2_id_map.get(candidate_name)
                best_fuzzy_score = fuzzy_score

    if llm_determined_match_name and highest_llm_score >= LLM_CONFIDENCE_THRESHOLD:
        crosswalk_results.append({
            "Table1_ID": id1,
            "Table1_Name": name1,
            "Table2_ID_Match": llm_determined_match_id,
            "Table2_Name_Match": llm_determined_match_name,
            "Match_Method": "LLM Refined Fuzzy",
            "Fuzzy_Score": best_fuzzy_score,
            "LLM_Score": highest_llm_score
        })
    else:
        # If LLM didn't confirm or score too low, try to use the best fuzzy match if it's strong
        if best_fuzzy_match_name and best_fuzzy_score >= FUZZY_MATCH_THRESHOLD:
            crosswalk_results.append({
                "Table1_ID": id1,
                "Table1_Name": name1,
                "Table2_ID_Match": best_fuzzy_match_id,
                "Table2_Name_Match": best_fuzzy_match_name,
                "Match_Method": "Fuzzy Only (LLM low confidence)",
                "Fuzzy_Score": best_fuzzy_score,
                "LLM_Score": highest_llm_score
            })
        else:
            crosswalk_results.append({
                "Table1_ID": id1,
                "Table1_Name": name1,
                "Table2_ID_Match": None,
                "Table2_Name_Match": None,
                "Match_Method": "No Match",
                "Fuzzy_Score": None,
                "LLM_Score": None
            })

# --- Display Results ---
results_df = pd.DataFrame(crosswalk_results)
print("\n--- Crosswalk Results ---")
print(results_df.to_string())

print("\n--- Summary ---")
matched_count = results_df['Table2_Name_Match'].count()
print(f"Total entries in Table 1: {len(table1_names)}")
print(f"Total matched entries: {matched_count}")
print(f"Unmatched entries: {len(table1_names) - matched_count}")


--- Crosswalk Results ---
      Table1_ID                                                                                                            Table1_Name  Table2_ID_Match                                             Table2_Name_Match                     Match_Method  Fuzzy_Score  LLM_Score
0          9211                                                                                     ROBERT W. BAIRD & CO. INCORPORATED          49552.0                                         ROBERT W BAIRD CO INC                LLM Refined Fuzzy         82.0       0.85
1          9211                                                                                              ROBERT W. BAIRD & CO. Inc          74372.0                                     ROBERT W BAIRD AND CO INC                LLM Refined Fuzzy         91.0       0.98
2         12518                                                                                         WILLIAM BLAIR & COMPANY L.L.C.          26558.0         

In [30]:
results_df

,Table1_ID,Table1_Name,Table2_ID_Match,Table2_Name_Match,Match_Method,Fuzzy_Score,LLM_Score
0,9211,ROBERT W. BAIRD & CO. INCORPORATED,49552.0,ROBERT W BAIRD CO INC,LLM Refined Fuzzy,82.0,0.85
1,9211,ROBERT W. BAIRD & CO. Inc,74372.0,ROBERT W BAIRD AND CO INC,LLM Refined Fuzzy,91.0,0.98
2,12518,WILLIAM BLAIR & COMPANY L.L.C.,26558.0,WILLIAM BLAIR AND CO LLC,LLM Refined Fuzzy,75.0,0.90
3,12933,"BOENNING & SCATTERGOOD, INC.",36825.0,BOENNING AND SCATTERGOOD INC,LLM Refined Fuzzy,92.0,0.98
4,18026,"CARTY & COMPANY, INC.",NaN,None,No Match,NaN,NaN
...,...,...,...,...,...,...,...
1023,1510788,"ENTREX CAPITAL MARKET, LLC",71289.0,JD CAPITAL MARKETS LLC,LLM Refined Fuzzy,81.0,0.80
1024,1620194,CHALDON ASSOCIATES LLC,59599.0,BLK ASSOCIATES LLC,Fuzzy Only (LLM low confidence),80.0,0.70
1025,1620468,"Providence Engineering & Design, LLC",NaN,None,No Match,NaN,NaN
1026,1623229,"Catlin Financial, LLC",68633.0,CATLIN FINANCIAL LLC,LLM Refined Fuzzy,100.0,1.00


In [32]:
results_df['Match_Method'].value_counts()

Match_Method
LLM Refined Fuzzy                  722
No Match                           275
Fuzzy Only (LLM low confidence)     31
Name: count, dtype: int64

In [31]:
results_df.to_csv(r'/Users/sukanyadatta/bm_code/muni_adv_pbf/Results/Step6c_df_SECtoMergent.csv', index=False)

In [18]:

# --- Configuration ---
# OLLAMA_MODEL = "mistral"  # Ensure you have 'mistral' pulled in Ollama
OLLAMA_MODEL = "deepseek-r1"  # Ensure you have 'deepseek-r1' pulled in Ollama
FUZZY_MATCH_THRESHOLD = 75 # Initial fuzzy matching threshold (0-100)
LLM_CONFIDENCE_THRESHOLD = 0.8 # Threshold for LLM's confidence score (0.0-1.0)
MAX_LLM_CANDIDATES = 5 # How many top fuzzy matches to send to the LLM for re-evaluation


# --- Helper Function for Name Normalization ---
def normalize_name(name: str) -> str:
    """
    Normalizes a string by converting to lowercase, removing common business suffixes,
    and extra spaces.
    """
    normalized = name.lower()
    # Remove common legal/business suffixes and punctuation
    suffixes = [
        r'\s*llc\.?', r'\s*inc\.?', r'\s*co\.?', r'\s*corp\.?',
        r'\s*ltd\.?', r'\s*group\.?', r'\s*capital\.?', r'\s*markets\.?',
        r'\s*corporation\.?'
    ]
    for suffix in suffixes:
        normalized = re.sub(suffix, '', normalized)
    
    # Remove any non-alphanumeric characters except spaces
    normalized = re.sub(r'[^a-z0-9\s]', '', normalized)
    # Replace multiple spaces with a single space and strip leading/trailing spaces
    normalized = re.sub(r'\s+', ' ', normalized).strip()
    return normalized

# --- Ollama Interaction Function ---
def get_llm_match_score(name1: str, name2: str) -> float:
    """
    Uses the Ollama LLM (Mistral) to determine a similarity score between two names.
    The LLM is asked to output a JSON object with a 'match_score'.
    The prompt is designed to make the LLM act more like a human agent.
    """
    prompt = f"""You are an expert at identifying similar entity names, even with spelling errors or slight variations.
    Your goal is to determine if two names refer to the *same underlying entity*.

    Rate their similarity on a scale from 0.0 to 1.0, where:
    - 1.0 is a perfect match (e.g., "Google Inc." and "Alphabet Inc." if you know they are the same, or "John Doe" and "John Doe").
    - 0.0 is no similarity.

    Consider the following:
    - Focus primarily on the *unique identifying words* or core parts of the names.
    - Common descriptive terms or legal suffixes (e.g., "LLC", "Capital Markets", "Group", "Corp") should be given less weight if the primary names are different.
    - Account for common spelling mistakes (e.g., "Jon Do" vs "John Doe") and variations.
    - If the core entities are clearly distinct, even if they share common industry terms, the score should be low.

    Return your answer as a JSON object with a single key 'match_score'.
    Example: {{ "match_score": 0.95 }}

    Name 1: "{name1}"
    Name 2: "{name2}"
    """
    
    # Implement exponential backoff for API calls
    retries = 3
    for i in range(retries):
        try:
            response = ollama.chat(
                model=OLLAMA_MODEL,
                messages=[{'role': 'user', 'content': prompt}],
                options={'temperature': 0.1}, # Keep temperature low for deterministic output
                format='json' # Request JSON output
            )
            # Parse the JSON response
            import json
            if 'message' in response and 'content' in response['message']:
                content = response['message']['content']
                try:
                    score_data = json.loads(content)
                    return float(score_data.get('match_score', 0.0))
                except json.JSONDecodeError:
                    print(f"Warning: LLM did not return valid JSON for '{name1}' vs '{name2}'. Response: {content}")
                    return 0.0 # Default to no match if JSON is invalid
            else:
                print(f"Warning: Unexpected LLM response structure for '{name1}' vs '{name2}'. Response: {response}")
                return 0.0
        except ollama.ResponseError as e:
            if e.status_code == 404:
                print(f"Error: Model '{OLLAMA_MODEL}' not found. Please pull it using 'ollama pull {OLLAMA_MODEL}'.")
                return 0.0
            elif e.status_code == 503 and i < retries - 1: # Service Unavailable, retry
                wait_time = (2 ** i) # Exponential backoff
                print(f"LLM call failed (status 503), retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"Error communicating with Ollama: {e}")
                return 0.0
        except Exception as e:
            print(f"An unexpected error occurred during LLM call: {e}")
            return 0.0
    return 0.0 # Return 0 if all retries fail


# --- Perform Crosswalk ---
crosswalk_results = []

# Extract names for fuzzy matching. Use normalized names for better initial candidate generation.
table2_names_normalized = [normalize_name(entry["name"]) for entry in table2_names]
# Map normalized name back to original name and ID for results
table2_normalized_to_original_map = {normalize_name(entry["name"]): {"original_name": entry["name"], "id": entry["id"]} for entry in table2_names}


for entry1 in table1_names:
    original_name1 = entry1["name"]
    id1 = entry1["id"]
    name1_normalized = normalize_name(original_name1)

    best_fuzzy_match_normalized_name = None
    best_fuzzy_match_original_name = None
    best_fuzzy_match_id = None
    best_fuzzy_score = -1

    # Step 1: Initial fuzzy matching to get top candidates using normalized names
    # We pass the normalized names for the fuzzy matching process
    fuzzy_matches = process.extract(name1_normalized, table2_names_normalized, scorer=fuzz.token_sort_ratio, limit=MAX_LLM_CANDIDATES)

    llm_determined_match_original_name = None
    llm_determined_match_id = None
    highest_llm_score = 0.0

    if not fuzzy_matches:
        print(f"No fuzzy matches found for '{original_name1}' (ID: {id1}).")
        crosswalk_results.append({
            "Table1_ID": id1,
            "Table1_Name": original_name1,
            "Table2_ID_Match": None,
            "Table2_Name_Match": None,
            "Match_Method": "No Match",
            "Fuzzy_Score": None,
            "LLM_Score": None
        })
        continue

    # Step 2: Re-evaluate top fuzzy matches using LLM for semantic understanding
    for candidate_normalized_name, fuzzy_score in fuzzy_matches:
        if fuzzy_score >= FUZZY_MATCH_THRESHOLD:
            # Get original name from the map for LLM evaluation, as LLM might benefit from original context
            candidate_original_info = table2_normalized_to_original_map.get(candidate_normalized_name)
            if candidate_original_info:
                candidate_original_name = candidate_original_info["original_name"]
                candidate_id = candidate_original_info["id"]

                # Pass original names to LLM for richer context
                llm_score = get_llm_match_score(original_name1, candidate_original_name)
                
                if llm_score > highest_llm_score:
                    highest_llm_score = llm_score
                    llm_determined_match_original_name = candidate_original_name
                    llm_determined_match_id = candidate_id
                    best_fuzzy_match_original_name = candidate_original_name # Store the fuzzy match that led to the best LLM score
                    best_fuzzy_match_id = candidate_id
                    best_fuzzy_score = fuzzy_score # Store the fuzzy score for this best candidate

    if llm_determined_match_original_name and highest_llm_score >= LLM_CONFIDENCE_THRESHOLD:
        crosswalk_results.append({
            "Table1_ID": id1,
            "Table1_Name": original_name1,
            "Table2_ID_Match": llm_determined_match_id,
            "Table2_Name_Match": llm_determined_match_original_name,
            "Match_Method": "LLM Refined Fuzzy",
            "Fuzzy_Score": best_fuzzy_score,
            "LLM_Score": highest_llm_score
        })
    else:
        # If LLM didn't confirm or score too low, try to use the best fuzzy match if it's strong
        # This fallback also uses the original name and ID that corresponded to the best fuzzy match
        if best_fuzzy_match_original_name and best_fuzzy_score >= FUZZY_MATCH_THRESHOLD:
            crosswalk_results.append({
                "Table1_ID": id1,
                "Table1_Name": original_name1,
                "Table2_ID_Match": best_fuzzy_match_id,
                "Table2_Name_Match": best_fuzzy_match_original_name,
                "Match_Method": "Fuzzy Only (LLM low confidence)",
                "Fuzzy_Score": best_fuzzy_score,
                "LLM_Score": highest_llm_score
            })
        else:
            crosswalk_results.append({
                "Table1_ID": id1,
                "Table1_Name": original_name1,
                "Table2_ID_Match": None,
                "Table2_Name_Match": None,
                "Match_Method": "No Match",
                "Fuzzy_Score": None,
                "LLM_Score": None
            })

# --- Display Results ---
results_df = pd.DataFrame(crosswalk_results)
print("\n--- Crosswalk Results ---")
print(results_df.to_string())

print("\n--- Summary ---")
matched_count = results_df['Table2_Name_Match'].count()
print(f"Total entries in Table 1: {len(table1_names)}")
print(f"Total matched entries: {matched_count}")
print(f"Unmatched entries: {len(table1_names) - matched_count}")

Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']



--- Crosswalk Results ---
      Table1_ID                                                                                                            Table1_Name  Table2_ID_Match                                             Table2_Name_Match                     Match_Method  Fuzzy_Score  LLM_Score
0          9211                                                                                     ROBERT W. BAIRD & CO. INCORPORATED              NaN                                                          None                         No Match          NaN        NaN
1          9211                                                                                              ROBERT W. BAIRD & CO. Inc          69006.0                                         ROBERT W BAIRD CO LLC                LLM Refined Fuzzy        100.0       0.80
2         12518                                                                                         WILLIAM BLAIR & COMPANY L.L.C.              NaN         

In [19]:
results_df['Match_Method'].value_counts()

Match_Method
LLM Refined Fuzzy                  673
No Match                           310
Fuzzy Only (LLM low confidence)     45
Name: count, dtype: int64

In [20]:
# results_df.to_csv(r'/Users/sukanyadatta/bm_code/muni_adv_pbf/Results/Step6c_df_SECtoMergent2.csv', index=False)
# switch to deepseek
results_df.to_csv(r'/Users/sukanyadatta/bm_code/muni_adv_pbf/Results/Step6c_df_SECtoMergent3_dsk.csv', index=False)